# Linear Regression

In this homework we are going to apply linear regression to two different problems. We'll begin by guiding you through predicting job satisfaction and the desire to be a manager among developers based on survey data. Once that's done, you will model candy preference based on composition and food science properties

In [1]:
import csv
import gzip
import math
import hashlib
import numpy as np
from pprint import pprint

from testing.testing import test

## Developers! Developers! Developers!

The data from this question is based on the [2019 StackOverflow Survey](https://insights.stackoverflow.com/survey/2019); accordingly, the subset bundled with this assignment is also released under the Open Database License (ODbL) v1.0.

The data was made by selecting some columns from the original dataset, only retaining rows from people who described themselves as "a developer by profession", and replaced long responses with shorter strings. Lets begin by examining the data.

In [2]:
def read_csv_test(read_csv):
    headers, rows = read_csv()
    test.equal(len(rows), 65679)
    test.equal(len(headers), 26)

    # Print a row:
    pprint(dict(zip(headers, rows[0])))
    
@test
def read_csv(fn="eggs.csv.gz"):
    """read the GZipped CSV data and split it into headers and newlines.
    
    kwargs:
        fn : str -- .csv.gz file to read
    
    returns: Tuple[headers, body] where
      headers : Tuple[str] -- the CSV headers
      body : List[Tuple[str,...]] -- the CSV body
    """
    with gzip.open(fn, 'rt', newline="", encoding='utf-8') as f:
        csvobj = csv.reader(f)
        headers = next(csvobj)
        return headers, [tuple(row) for row in csvobj]


{'Age': '22',
 'CareerSat': 'vs',
 'CodeRevHrs': 'NA',
 'ConvertedComp': '61000',
 'Country': 'United States',
 'Dependents': 'n',
 'DevEnvironVSC': 'y',
 'DevTypeFullStack': 'n',
 'EdLevel': 'bachelors',
 'EduOtherMOOC': 'y',
 'EduOtherSelf': 'y',
 'Extraversion': 'y',
 'GenderIsMan': 'y',
 'Hobbyist': 'n',
 'MgrIdiot': 'very',
 'MgrWant': 'n',
 'OpSys': 'win',
 'OpenSourcer': 'never',
 'OrgSize': '100-499',
 'Respondent': '4',
 'Student': 'n',
 'UndergradMajorIsComputerScience': 'y',
 'UnitTestsProcess': 'n',
 'WorkWeekHrs': '80',
 'YearsCode': '3',
 'YearsCodePro': '0'}
### TESTING read_csv: PASSED 2/2
###



In [3]:
# headers, data = read_csv()
# headers, data = np.array(headers), np.array(data)
# wholeData = np.vstack((headers, data))
# import pandas as pd
# df = pd.DataFrame(wholeData)
# new_header = df.iloc[0]
# df.columns = new_header
# map(type_CareerSat, df['CareerSat'].values)
# df.head()

Our task is to predict:

1. if respondents are managers or want to be a manager in the future (`MgrWant` is `y`), and
2. if respondents are satisfied with their career (`CareerSat` is `vs` or `ss`)

based on the remaining rows. We have bolded these rows in the table below.

Before we can use linear regression, we must convert this into numeric data. This is the core challenge of this problem; Here's a table of rows and what they mean:


| Column | Sample | Does/is the respondent... | Type/Values |
| --- |:--- |:--- |:--- |
| **CareerSat** | 'vs' | satisfied with their career? | (`vd`, `sd`, `ne`, `ss`, `vs`) -- corresponding to ({very, slightly}, {satisfied, dissatisfied}) and neutral |
| **MgrWant** | 'n' | ...want to be a manager? | boolean |
| Age    | '22'   | age | integer     |
| CodeRevHrs | '2' | hours a week spent reviewing code | integer |
| ConvertedComp | '61000' | yearly compensation in 2019 USD | integer |
| Country | 'United States' | lives in country | string _(ignore in regression)_ |
| Dependents | 'n' | ...have children or other dependents. | boolean |
| DevEnvironVSC | 'y' | ...use Visual Studio Code | boolean |
| DevTypeFullStack | 'n' | ...identify as a full-stack developer | boolean |
| EdLevel | 'bachelors' | maximum education level | (`other`, `bachelors`, `masters`, `doctoral`) |
| EduOtherMOOC | 'y' | ...ever taken a Massively Open Online Course | boolean |
| EduOtherSelf | 'y' | ...ever taught themselves a new platform | boolean |
| Extraversion | 'y' | ...prefer in-person meetings to online meetings | boolean |
| GenderIsMan | 'y' | ...male | boolean |
| Hobbyist | 'n' | ...write code as a hobby? | boolean |
| MgrIdiot | 'very' | ...think their manager knows what they are doing? | (`NA`, `not`, `some`, `very`), in order of increasing confidence |
| OpSys | 'win' | which OS do they use? | (`win`, `mac`, `tux`, `NA`), for (Windows, Mac OSX, Linux-like, NA) |
| OpenSourcer | 'Never' | ...contribute to open-source projects? | (`never`, `year`, `month-year`, `month`), in increasing order of frequency |
| OrgSize | '100-499' | number of employees in organization? | (`NA`, `1`, `2-9`, `10-19`, `20-99`, `100-499`, `500-999`, `1,000-4,999`, `5,000-9,999`, `10,000+`) |
| Respondent | '4' | respondent ID from original data | integer _(ignore in regression)_ |
| Student | 'n' | ...currently a student? | boolean |
| UndergradMajorIsComputerScience | 'y' | ...majored in CS? | boolean |
| UnitTestsProcess | 'n' | ...use unit tests in their job? | boolean |
| WorkWeekHrs | '80' | hours a week worked | integer |
| YearsCode | 3 | years since first programming | integer |
| YearsCodePro | 0 | years programming professionally | integer |

### Type conversion

Now for the slow data-cleaning grind that is characteristic of work as a data scientist. We begin by writing type coercion functions: functions that convert each column type into a `float` value for use in linear regression. All input values are `str`.

The column types are:

 - _boolean_ : `y`/`NA`/`n` assigned to `+1.0`/`0.0`/`0.0`
 - _integer_ : convert to `float`, preserving value. `NA` equals `0.0`. 
 - _string_ : not included in regression; we'll use it later
 - CareerSat: Map (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) to (-2.0, -1.0, 0.0, 0.0, 1.0, 2.0)
 - EdLevel: Map (`other`, `bachelors`, `masters`, `doctoral`) to (0.0, 1.0, 1.5, 2.0)
 - MgrIdiot: Map (`NA`, `not`, `some`, `very`) to (-1.0, -1.0, 0.0, 1.0)
 - OpSys: This is a category variable, we will split this into three columns (one for each possible value) and set 1.0 in the corresponding column. This is called a [one-hot encoding](https://en.wikipedia.org/wiki/One-hot). (Don't write a conversion function in this step.)  00001，00010
 - OpenSourcer : Map (`never`, `year`, `month-year`, `month`) to (0.0, 0.5, 1.0, 2.0)
 - OrgSize: Map each range "$a$-$b$" to the value $ln(a)$. Treat `NA` as `ln(1.0) = 0`. We are converting an exponentially distributed range to a linearly distributed one.

All your conversion functions must throw an exception if you encounter an unexpected value. As an example, we give you the boolean conversion function:

In [4]:
def type_boolean_test(type_boolean):
    test.true(isinstance(type_boolean("y"), float))
    test.equal(type_boolean("y"), 1.0)
    test.equal(type_boolean("n"), 0.0)
    test.exception(lambda: type_boolean("5"))

@test
def type_boolean(c):
    if c == "y": return 1.0
    elif c == "n": return 0.0
    elif c == "NA": return 0.0
    raise ValueError(c)

### TESTING type_boolean: PASSED 4/4
###



Now fill in these functions according to specification:

In [5]:
# Integer
def type_integer_test(type_integer):
    test.true(isinstance(type_integer("5"), float))
    test.equal(type_integer("3"), 3.0)
    test.equal(type_integer("0"), 0.0)
    test.equal(type_integer("-4"), -4.0)
    test.equal(type_integer("NA"), 0.0)
    test.exception(lambda: type_integer("yes"))

@test
#what we want is to convert any int type variables into float, and if the variable is NA we set it to 0.0
def type_integer(c):
    if c == "NA":
        return 0.0
    else :
        return float(c)


# CareerSat
def type_CareerSat_test(type_CareerSat):
    test.true(isinstance(type_CareerSat("vd"), float))
    test.equal(type_CareerSat("sd"), -1.0)
    test.equal(type_CareerSat("ne"), 0.0)
    test.equal(type_CareerSat("ss"), 1.0)
    test.equal(type_CareerSat("vs"), 2.0)
    test.exception(lambda: type_CareerSat("yes"))

@test
def type_CareerSat(c):
    mapping = {"vd":-2.0, "sd":-1.0,
              "ne" : 0.0, "NA":0.0,
              "ss":1.0, "vs":2.0}
    return mapping[c]


# EdLevel
def type_EdLevel_test(type_EdLevel):
    test.true(isinstance(type_EdLevel("other"), float))
    test.equal(type_EdLevel("bachelors"), 1.0)
    test.equal(type_EdLevel("masters"), 1.5)
    test.equal(type_EdLevel("doctoral"), 2.0)
    test.exception(lambda: type_EdLevel("yes"))

@test
def type_EdLevel(c):
    mapping = {"other" : 0.0,  "bachelors": 1.0, 
               "masters": 1.5, "doctoral": 2.0}
    return mapping[c]


# MgrIdiot
def type_MgrIdiot_test(type_MgrIdiot):
    test.true(isinstance(type_MgrIdiot("NA"), float))
    test.equal(type_MgrIdiot("not"), -1.0)
    test.equal(type_MgrIdiot("some"), 0.0)
    test.equal(type_MgrIdiot("very"), 1.0)
    test.exception(lambda: type_MgrIdiot("yes"))

@test
def type_MgrIdiot(c):
    mapping = {"NA" : -1.0, "not": -1.0, "some": 0.0, "very": 1.0}
    return mapping[c]



# OpenSourcer
def type_OpenSourcer_test(type_OpenSourcer):
    test.true(isinstance(type_OpenSourcer("never"), float))
    test.equal(type_OpenSourcer("year"), 0.5)
    test.equal(type_OpenSourcer("month-year"), 1.0)
    test.equal(type_OpenSourcer("month"), 2.0)
    test.exception(lambda: type_OpenSourcer("yes"))

@test
def type_OpenSourcer(c):
    mapping = {"never" : 0.0, "year" : 0.5, "month-year" : 1.0, "month" : 2.0}
    return mapping[c]


# OrgSize
def type_OrgSize_test(type_OrgSize):
    test.true(isinstance(type_OrgSize("1"), float))
    test.equal(type_OrgSize("NA"), 0)
    test.equal(type_OrgSize("2-9"), 0.6931471805599453)
    test.equal(type_OrgSize("100-499"), 4.605170185988092)
    test.equal(type_OrgSize("10,000+"), 9.210340371976184)
    test.exception(lambda: type_OrgSize("yes"))

@test
def type_OrgSize(c):
    '''input arg: string, "100-200" '''
    if c == "NA":
        return np.log(1)
    c = c.replace("+","").replace(",","")
    try:
        whereStop = c.index("-")
        digit = c[:whereStop]
    except:
        try:
            if c.isnumeric():
                digit = int(c)
        except:
            raise ValueError('Invalid Input')

    return np.log(int(digit))

### TESTING type_integer: PASSED 6/6
###

### TESTING type_CareerSat: PASSED 6/6
###

### TESTING type_EdLevel: PASSED 5/5
###

### TESTING type_MgrIdiot: PASSED 5/5
###

### TESTING type_OpenSourcer: PASSED 5/5
###

### TESTING type_OrgSize: PASSED 6/6
###



Now we use these to convert the data into floating-point numbers. 

This is also where we deal with `OpSys`; from the one column in the source, create three columns (called `OpSysWin`, `OpSysMac`, and `OpSysTux`, corresponding to the values `win`, `mac`, `tux`. Other values can be ignored.) For each row, at most one of the cells must be 1.0, and the others must be 0.0. If the value in the cell is `NA`, then all the cells must be 0.0.

This is called a [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) and is a common way to handle category variables.

In [6]:
# a = ["a", "b"]
# "b" in a

In [7]:
def quick_checksum(st):
    return hashlib.md5((" ".join(sorted(st))).encode()).hexdigest()

def convert_data_stackoverflow_test(convert_data_stackoverflow):
    headers, rows = convert_data_stackoverflow(*read_csv())
    # If this test fails, your headers are incorrect:
    test.equal(quick_checksum(headers), "5f9fc24f9e8dae961c2a778f493940ed")
    # Type check:
    test.true(all(all(isinstance(v, float) for v in r) for r in rows))
    # Operating System columns:
    for row in rows:
        d = dict(zip(headers, row))
        if sorted([d["OpSysWin"], d["OpSysMac"], d["OpSysTux"]]) not in [[.0, .0, 1.], [0.]*3]:
            test.true(False)
            break
    else:
        test.true("There is correctly at most one OpSys* column set to 1.0")
    

@test
def convert_data_stackoverflow(headers, data):
    """convert the data into 
    
    args:
        header : List[str] -- the header for each column in the CSV
        data : List[Tuple[str]] -- the CSV data, where each inner list corresponds to a row in the CSV file.
 
    returns: Tuple[headers, body] where
      headers : List[str] -- the new headers, dropping the Country and Respondent headers and expanding 
      body : List[List[str,...]] -- the CSV body
    """
    #first we need to check if the original data width is the same as the length of ther header\
    assert len(headers) == len(data[0])
    
    #needed to be dropped:
    dropping = ["Country", "Respondent", "headers"]
    
    #we need to track which index is indicating which column name 
    whichColumn = {} 
    for i in range(len(headers)):
        whichColumn[i] = headers[i]
        
    #method map, as different column has different method; also not including the drpped and 'OpSys'
    #the 'OpSys' needs to be processed into on hot vector
    whichMethod = {'CareerSat': type_CareerSat, "MgrWant": type_boolean, "Hobbyist":type_boolean,
                  'OpenSourcer': type_OpenSourcer, "Student" : type_boolean, "EdLevel": type_EdLevel,
                  'UndergradMajorIsComputerScience': type_boolean, 'EduOtherMOOC': type_boolean,
                  'OrgSize': type_OrgSize, 'DevTypeFullStack': type_boolean, 'YearsCode': type_integer,
                  'MgrIdiot': type_MgrIdiot, 'ConvertedComp' : type_integer, 'WorkWeekHrs' : type_integer,
                  'CodeRevHrs': type_integer, 'UnitTestsProcess': type_boolean, 'Extraversion': type_boolean,
                  'Age': type_integer, 'GenderIsMan': type_boolean, 'Dependents': type_boolean,
                  'OpSys':'oneHot', 'EduOtherSelf':type_boolean, 'YearsCodePro': type_integer,
                  'DevEnvironVSC': type_boolean}
    
    #one hot decode
    oneHotHash = {'win':[1.0, 0.0, 0.0], 'mac' : [0.0, 1.0, 0.0], 'tux': [0.0, 0.0, 1.0]}
    
    #Now all the hash is
    #one hot vector, which inherently would introduce 3 more columns. Also, this also means that if the 'whichMethod' hash does
    #not contain the column name, that just means that it is the 'OpSys'
    new_header = []
    for col in headers:
        if col != "Country" and  col != "Respondent" and col != "OpSys":
            new_header.append(col)
    new_header.extend(["OpSysWin", "OpSysMac", "OpSysTux"])
    
    newDataReturn = []
    for rowData in data:
        newRowData = []
        for i in range(len(data[0])):
            #we do not need to process the data under the dropped column
            if whichColumn[i] in dropping:
                continue
            thisColumn = whichColumn[i]
            thisMethod = whichMethod[thisColumn]
            if thisMethod == 'oneHot':
                if rowData[i] in oneHotHash:
                    oneHot = oneHotHash[rowData[i]]
                else:
                    oneHot = [0.0,0.0,0.0]
            else:       
                newData = thisMethod(rowData[i])
                newRowData.append(newData)    
        newRowData.extend(oneHot)
        newDataReturn.append(newRowData)
        
    return (new_header, newDataReturn)

### TESTING convert_data_stackoverflow: PASSED 3/3
###



### Splitting Data

Now we prepare the converted data for regression. In this step, we:

 1. split this into training and validation sets,
 2. convert it to a Numpy `ndarray` with underlying type `np.float32`,
 3. split each set into the predicted columns and the feature columns.

We will save the first 20% of the dataset (rounded down) as the validation set and keep the remaining as the training set. (Note that it is common practice to randomize the dataset; this has already been done. Don't shuffle the dataset for this assignment.)

Ensure that the underlying type of the `ndarray` is `np.float32`, not the default `np.float64`. We do not need the added precision of 64-bit floating point numbers for this problem, and using the smaller numbers will speed up computation and reduce the amount of memory we need.

In [8]:
def split_data_test(split_data):
    headers, rows = convert_data_stackoverflow(*read_csv())
    l = len(rows)
    
    val, train = split_data(rows)
    test.equal(len(val), l // 5)
    test.true(isinstance(val, np.ndarray))
    test.equal(val.dtype, np.float32)
    test.equal(len(train), l - (l // 5))
    test.true(isinstance(train, np.ndarray))
    test.equal(train.dtype, np.float32)

@test
def split_data(data):
    """split the data into training and validation sets, and convert them to np.ndarray. (Step 1 and 2 above.)

    args:
        data : List[List[str]] -- the CSV data, where each inner list corresponds to a row in the CSV file.

    returns: Tuple[val, train] where
      val  : np.ndarray[num_val_rows, num_features] -- the first 20% of the dataset (rounded down)
      train : np.ndarray[num_train_rows, num_features] -- the remaining rows from data
    
    Ensure that the underlying type of the output is np.float32, not the default np.float64.
    """
    valLength = int(len(data) * 0.2)
    valSet = np.asarray(data[:valLength], dtype = np.float32)
    trainSet = np.asarray(data[valLength:], dtype = np.float32)
    return(valSet, trainSet)
#     pass

### TESTING split_data: PASSED 6/6
###



In [9]:
def separate_objective_test(separate_objective):
    headers, rows = convert_data_stackoverflow(*read_csv())
    val, train = split_data(rows)

    for subset in [val, train]:
        subset_headers, subset_features, subset_objectives = separate_objective(headers, subset, ["CareerSat", "MgrWant"])

        test.true(isinstance(subset_objectives, tuple))
        test.equal(len(subset_objectives), 2)
        test.true("CareerSat" not in subset_headers)
        test.true("MgrWant" not in subset_headers)
        test.equal(subset_features.shape[1], 24)

@test
def separate_objective(headers, data, objectives):
    """split the objective columns from the headers and data. (Step 1 and 2 above.)

    args:
        headers    : List[str] -- the headers for the data, used to find the objective columns from the data array
        data       : np.ndarray[num_rows, num_columns] -- the data
        objectives : the columns to extract from the data, list of strings

    returns: Tuple[o_headers, o_features, o_objectives] where
      o_headers  : List[str] -- a list of headers without the objective columns
      o_features : np.ndarray[num_train_rows, num_features] -- the remaining columns from data. (num_features = num_columns - len(objectives))
      o_objectives : Tuple[np.ndarray[num_train_rows], ...] -- a list of objective columns from the data, each element is a 1-dimensional np.ndarray corresponding to the entry in objectives.
     """
    #the needed deleted items are saved in the objectives as a List<String>
    o_headers = []    #the headers without the objective columns 
    whichColumn = set() #we need to track the deleted column index
    
    for idx, title in enumerate(headers):
        if title in objectives:
            whichColumn.add(idx)
        else:
            o_headers.append(title)
    
    #now we need to extact out the needed features   m by n, m samples, n features
    #and we need the labels, m by nLabels
    o_features = []
    o_objectives = []
    for row in data:
        target_data = []
        features = []
        for i in range(len(row)):
            #if this idx is in the removed hash, we know it is the label
            if i in whichColumn:
                target_data.append(row[i])
            else:
                features.append(row[i])
        o_features.append(features)
        o_objectives.append(target_data)
        
#     print("labels in ", np.array(o_objectives).T.shape)
#     print("features in ", np.array(o_features).shape)
    o_objectives = tuple(np.array(o_objectives).T)
    o_features = np.array(o_features)

    return (o_headers, o_features, o_objectives)
    
    

### TESTING separate_objective: PASSED 10/10
###



### Linear Regression

Now you will finally implement a linear regression. As a reminder, linear regression models the data as

$$\mathbf y = \mathbf X\mathbf \beta + \mathbf \epsilon$$

where $\mathbf y$ is a vector of outputs, $\mathbf X$ is also known as the design matrix, $\mathbf \beta$ is a vector of parameters, and $\mathbf \epsilon$ is noise. We will be estimating $\mathbf \beta$ using Ordinary Least Squares, and we recommending following the matrix notation for this problem (https://en.wikipedia.org/wiki/Ordinary_least_squares).

You are not allowed to use `scipy` in your submission for this assignment, but you are encouraged to use it to test your solution. Make sure that you only ever `import scipy` inside a `_test` function.

Hints:

 1. You should use `np.linalg.solve` to calculate `beta`.
 2. Feel free to add `1e-4*np.eye(...)` to the coefficient matrix

In [10]:
class LinearRegression():
    """ Perform linear regression and predict the output on unseen examples. 
    
    attributes: 
        beta (np.ndarray) : vector containing parameters for the features
    """

    def train(self, X, y):
        """ Train the linear regression model by computing the estimate of the parameters
        You should store the model parameters in self.beta, overwriting parameters as necessary.

        args: 
            X (np.ndarray[num_examples, num_columns]) : matrix of training data
            y (np.ndarray[num_examples]) : vector of output variables

        return: LinearRegression -- returns itself (for convenience)
        """
        #extract the m samples and n features out 
        m,n = X.shape
        #adding the 1e-4
        X = X + 1e-4 * np.eye(m,n)
        #construct the matrix
        A = np.zeros((n,n))
        for i in range(m):
            A += np.outer(X[i],X[i])
           
        #convert the y into the same dimension - > mSampe * 1
        yy = np.expand_dims(y, 1)
        #element wise production and sum all the rows 
        b = sum(X * yy)
        self.beta = np.linalg.solve(A,b)

        return self

    def predict(self, X_p): 
        """ Use the learned model to predict the output of X_p

        args: 
            X_p (np.ndarray[num_examples, num_columns]) matrix of test/validation data where each row corresponds to an example

        return: 
            (np.ndarray[num_examples]) vector of predicted outputs
        """
        y_pred = X_p @ self.beta    # -> m by 1 predictions
        print(y_pred.shape)
        return y_pred

In [11]:
# Don't remove this function; we use it for the auto-grader.
def linear_regression_instance():
    return LinearRegression()

### Error Functions

One last part to this: linear regression minimizes the mean-squared-error. Write a function that calculates the mean mean-squared-error when given a prediction and a ground-truth vector.

In [12]:
def mean_squared_error_test(mean_squared_error):
    test.equal(mean_squared_error(np.ones(10), np.ones(10)), 0)
    test.equal(mean_squared_error(np.ones(10), np.zeros(10)), 1)

@test
def mean_squared_error(pred, ground_truth):
    """ calculate the mean mean-squared-error between pred and ground_truth
    
    args:
      pred : np.ndarray[num_examples] -- the predictions
      ground_truth : np.ndarray[num_examples] -- the ground truth values
      
    returns: float -- the average mean-squared-error between predictions and ground_truth values.
    """
    print("ground Truth shape", ground_truth.shape)
    print("prediction shape",pred.shape)
    MSE = np.square(np.subtract(ground_truth, pred)).mean()
    return MSE

ground Truth shape (10,)
prediction shape (10,)
ground Truth shape (10,)
prediction shape (10,)
### TESTING mean_squared_error: PASSED 2/2
###



### Putting it all together

And finally, lets run the entire pipeline end-to-end. You should put all the functions you have written so far together to:

1. read and split the dataset,
2. train two separate models on the training set, one to predict `MgrWant` and the other to predict `CareerSat`,
3. perform inference on the validation set, and
4. return the mean-squared error for each.

Remember not to include both columns `MgrWant` and `CareerSat` when training models to predict either column. (i.e. when training `MgrWant`, you should not include `CareerSat` and vice-versa.)

In [13]:
def linear_regression_run_test(linear_regression_run):
    mse_mgr, mse_sat = linear_regression_run(*read_csv())
    test.true(np.abs(mse_sat - 0.07214) < 1e-4)
    test.true(np.abs(mse_mgr - 1.29104) < 1e-4)

@test
def linear_regression_run(headers, rows):
    """ Perform linear regression on (headers, rows), and return the MSE on the validation set for both `MgrWant` and `CareerSat`. 

    args: 
        headers : List[str] -- headers from CSV file
        rows : np.ndarray[num_examples, num_columns] -- data from the CSV file
        
    return: Tuple[MSEMgrWant, MSECareerSat], where
        MSEMgrWant : float -- the MSE between the predictions and the ground truth values for the column `MgrWant`.
        MSECareerSat : float -- the MSE between the predictions and the ground truth values for the column `CareerSat`.
    """
    #fisrt step is to conver all text data and drop non- needed data points
    newHeader, data = convert_data_stackoverflow(headers, rows)
    
    #and then let us split the data into train and val set
    valSet, trainSet = split_data(data)
    
    #get the features and labels out
    trainHeader, trainFeatures, trainLabels = separate_objective(newHeader, trainSet, objectives = ["MgrWant","CareerSat"])
    valHeader, valFeatures, valLabels = separate_objective(newHeader, valSet, objectives = ["MgrWant","CareerSat"])
    
    #get the labels for mgr want and the careersat labels out 
    mrgLabelTrain, carLabelTrain = trainLabels[0], trainLabels[1]
    mrgLabelVal, carLabelVal = valLabels[0], valLabels[1]
    
    #build our two models
    mrgModel =linear_regression_instance()
    carModel = linear_regression_instance()
    
    #trainout two models
    mrgModel = mrgModel.train(trainFeatures, mrgLabelTrain)
    carModel = carModel.train(trainFeatures, carLabelTrain)
    
    #prediction
    mrgPred = mrgModel.predict(valFeatures)
    carPred = carModel.predict(valFeatures)
    
    #calculation
    mrgError = mean_squared_error(mrgPred, mrgLabelVal)
    carError = mean_squared_error(carPred, carLabelVal)

    return carError, mrgError

(13135,)
(13135,)
ground Truth shape (13135,)
prediction shape (13135,)
ground Truth shape (13135,)
prediction shape (13135,)
### TESTING linear_regression_run: PASSED 0/2
# 0	: Assertion failed
# 1	: Assertion failed
###



Note that `MgrWant` is a binary variable, with range $[-1, 1]$, and `CareerSat` has range $[-2, 2]$; relative to the range, the mean-squared error for `CareerSat` is much smaller than `MgrWant`. This means that we can better predict `CareerSat` than `MgrWant`. Great!

## Your Turn

Now that we've walked through this once with a large dataset, it is your turn to do this. You will be using [FiveThirtyEight's The Ultimate Halloween Candy Power Ranking](https://www.kaggle.com/fivethirtyeight/the-ultimate-halloween-candy-power-ranking/), included (and shuffled) as `candy.csv.gz`. (The dataset is Copyright (c) 2014 ESPN Internet Ventures. Our shuffled version is released under the MIT License.)

From the original documentation, here is a description of the columns:

| Column | Description | type |
| --- |:--- |:--- |
| **`winpercent`** | The overall win percentage according to 269,000 matchups. | float |
| `competitorname` | The bar name | string (don't use this) |
| `chocolate` | Does it contain chocolate? | boolean (`y`, `n`) |
| `fruity` | Is it fruit flavored? | boolean (`y`, `n`) |
| `caramel` | Is there caramel in the candy? | boolean (`y`, `n`) |
| `peanutalmondy` | Does it contain peanuts, peanut butter or almonds? | boolean (`y`, `n`) |
| `nougat` | Does it contain nougat? | boolean (`y`, `n`) |
| `crispedricewafer` | Does it contain crisped rice, wafers, or a cookie component? | boolean (`y`, `n`) |
| `hard` | Is it a hard candy? | boolean (`y`, `n`) |
| `bar` | Is it a candy bar? | boolean (`y`, `n`) |
| `pluribus` | Is it one of many candies in a bag or box? | boolean (`y`, `n`) |
| `sugarpercent` | The percentile of sugar it falls under within the data set. | float |
| `pricepercent` | The unit price percentile compared to the rest of the set. | float |


You must predict `winpercent` using exactly **four** other columns. Use the first 20% of the dataset as the validation set (the dataset has already been shuffled for you). As output, you should provide the names of the columns and the validation of the MSE. Your MSE must be no more than `330`.

You should convert boolean columns using `type_boolean`, and `LinearRegression` to perform the regression. Don't implement a bias term/constant column. Feel free to create new helper functions as necessary.

In [26]:
def candy_test(candy):
    headers, mse = candy(*read_csv("candy.csv.gz"))
    #print((headers, mse))
    test.true(len(headers) == 4)
    test.true("winpercent" not in headers)
    test.true(mse < 330.)

def conversion(dataPoint):
    
    try:
        dataPoint = type_boolean(dataPoint)
    except:
        dataPoint = type_integer(dataPoint)

    return dataPoint
                
@test
def candy(headers, data):
    """ predict winpercent using no more than four other columns
    
    args:
        headers : List[str] -- headers read from the csv file
        data : List[List[str]] -- data from the csv file

    returns: Tuple[selected_headers, mse]
        selected_headers : List[str] -- the headers of at most four columns used to train the model
        mse : float -- the mean-squared error when the columns in selected_headers are used to predict `winpercent`
    """
    print(headers)
    headerMap = {}
    for i in range(len(headers)):
        headerMap[i] = headers[i]
    
    dataSet = []
    label = []
    selectedHeaders = ['chocolate', 'fruity','sugarpercent','pricepercent']
    
    #now we want to get the trainning datapoints
    for row in data:
        rowData = []
        rowLabel = []
        for i in range(len(row)):
            thisColumn = headerMap[i]
            if thisColumn in selectedHeaders:
                rowData.append(conversion(row[i]))
            if thisColumn == 'winpercent':
                rowLabel.append(conversion(row[i]))
        dataSet.append(rowData)
         #this is important cuz we should not use append method, which would lead to [[],[],[]].
         #but we just want a list of numbers, not a list of list 
        label.extend(rowLabel)  
    
    valSet, trainSet = split_data(dataSet)
    valLabel, trainLabel = split_data(label)
    
    model = LinearRegression()
    model = model.train(trainSet, trainLabel)
    prediction = model.predict(valSet)
    
    error = mean_squared_error(prediction, valLabel)
    print("MSE Loss {}".format(error))
    return selectedHeaders, error

['competitorname', 'chocolate', 'fruity', 'caramel', 'peanutyalmondy', 'nougat', 'crispedricewafer', 'hard', 'bar', 'pluribus', 'sugarpercent', 'pricepercent', 'winpercent']
(68,)
ground Truth shape (68,)
prediction shape (68,)
MSE Loss 287.42907494476367
### TESTING candy: PASSED 3/3
###

